# Task 1: Housing Price Regression

Motivation here.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV
from sklearn.metrics import mean_squared_error

#import torch
#import torch.nn as nn
#import torch.optim as optim
#from torch.utils.data import Dataset,DataLoader

import re

In [2]:
# Hyperparameters

RANDOM_CONTROL = 42 # For reproducibility of notebook
TRAIN_SIZE = 0.8

# Random Forest: Fill in based on GridSearch results
RF_NUM_ESTIMATORS = 100
RF_MAX_DEPTH = 50
RF_MAX_FEATURES = 1
RF_MIN_SPLIT = 2
RF_MIN_LEAF = 1
RF_BOOTSTRAP = True
RF_CRITERION = "squared_error"

# Gradient Boosting: Fill in based on GridSearch results
GB_NUM_ESTIMATORS = 100
GB_MAX_DEPTH = 50
GB_CRITERION = "squared_error"

# AdaBoost: Fill in based on GridSearch results
AB_NUM_ESTIMATORS = 100
AB_MAX_DEPTH = 50
AB_LEARNING_RATE = 0.1

# Neural Net: Fill in based on test iterations
NN_NUM_EPOCHS = 10
NN_BATCH_SIZE = 32
NN_LEARNING_RATE = 0.1

In [3]:
# Read training data
df = pd.read_csv('data/train.csv') 

df.head(3)

df.shape

(20254, 21)

# EDA

Talk about pre-processing here.
Visualize plots of original data.

In [4]:
def visualize():
    pass

# IMPORTANT: Trivial modifications only. Do not aggregate/standardize/impute here!!

def ignore_attributes(df) -> pd.DataFrame:
    # Drop listing id; nominal identifier with no meaning
    df.drop('listing_id', axis=1, inplace=True)

    # Drop elevation; all the values are 0, spurious attribute
    df.drop('elevation', axis=1, inplace=True)

    # Drop url; nominal identifier with no meaning; useful for manual lookups or scraping
    df.drop('property_details_url', axis=1, inplace=True)

    # BELOW CODE IN THIS SECTION IS ONLY MEANT TO GET THE SKELETON WORKING; RE-EVALUATE EACH ATTRIBUTE ONE BY ONE
    df.drop('title', axis=1, inplace=True)
    df.drop('address', axis=1, inplace=True)
    df.drop('property_name', axis=1, inplace=True)
    df.drop('property_type', axis=1, inplace=True)
    df.drop('tenure', axis=1, inplace=True)
    df.drop('built_year', axis=1, inplace=True)
    df.drop('floor_level', axis=1, inplace=True)
    df.drop('furnishing', axis=1, inplace=True)
    df.drop('available_unit_types', axis=1, inplace=True)
    df.drop('total_num_units', axis=1, inplace=True) # Bernard verified dropping it. 27.9% missing.
    df.drop('lat', axis=1, inplace=True)
    df.drop('lng', axis=1, inplace=True)
    df.drop('subzone', axis=1, inplace=True)
    df.drop('planning_area', axis=1, inplace=True)

    return df

def handle_missing_values(df) -> pd.DataFrame:
    # Treat missing year data as new.
    # Semantically, we define this attribute as the depreciation factor for pricing.
    # A new house or one with missing data denotes the depreciation factor is 0 or unknown.
    # The depreciation factor is assumed to be the difference between construction and current year.
    # TODO: Maybe do not treat future years as current! Inflation factor might be one to look out for.
    df['built_year'] = df['built_year'].fillna(2022)
    
    # TODO: 80 are missing. Should we remove them or should we keep it as 0?
    # Verify assumption if studio qualifies as 1 bed. All 5 NaN were studios!
    df['num_beds'] = df['num_beds'].fillna(1) 

    # TODO: 400 are missing. Cannot remove so many data. Use 0 to denote absence of attribute.
    df['num_baths'] = df['num_baths'].fillna(0)
    
    
    return df
    
def handle_invalid_values(df) -> pd.DataFrame:
    # Price is the target regression variable. If negative or 0, treat that row as invalid.
    df = df[df.price > 0]

    # TODO: Verify the steps below for HDB - bed/bath ratio, price checks
    # Filtering those with number of bathrooms more than number of bedrooms for HDB
    filter_bath_beds_hdb = ((df.num_baths > df.num_beds) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df = df.drop(df[filter_bath_beds_hdb].index)

    #Filtering those with number of bathrooms more than 4, number of bedrooms more than 4 for HDB
    filter_bath_beds_4_hdb = (((df.num_baths > 4) | (df.num_beds > 5)) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df = df.drop(df[filter_bath_beds_4_hdb].index)
    
    # price ; filtering for HDB price more than 2 million
    filter_price_hdb = ((df.price > 2000000) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df = df.drop(df[filter_price_hdb].index)
        
    return df

def transform_data(df) -> pd.DataFrame:
    # TODO: Test against not doing this.
    df.loc[df["built_year"] > 2022, "built_year"] = 2022
    
    # Convert built_year into the aforementioned depreciation factor
    df["depreciation"] = (2022-df["built_year"])
    
    return df
    
def pre_process(df) -> pd.DataFrame:
    df = handle_missing_values(df)
    df = handle_invalid_values(df)
    df = transform_data(df)
    df = ignore_attributes(df)
    
    return df


df_preprocessed = pre_process(df)

In [5]:
print(df_preprocessed.shape)
df_preprocessed.head()

(20106, 5)


,num_beds,num_baths,size_sqft,price,depreciation
0,3.0,2.0,1115,514500.0,34.0
1,4.0,2.0,1575,995400.0,30.0
2,4.0,6.0,3070,8485000.0,0.0
3,3.0,2.0,958,2626000.0,0.0
4,2.0,1.0,732,1764000.0,0.0


In [6]:
# Utility functions



In [7]:
# Split data into train and validation set

X_housing = df_preprocessed.loc[:, df_preprocessed.columns != 'price']
y_housing = df_preprocessed['price']

X_train, X_val, y_train, y_val = train_test_split(X_housing, y_housing, train_size=TRAIN_SIZE, random_state=RANDOM_CONTROL, shuffle=True) 

# print(X_train.shape)
# print(y_train.shape)


# Perform further pre-processing steps only on train set

**Models**

(Add outline of steps here later)

- Linear Regression
- Random Forest
- Gradient Boosting
- AdaBoost
- Extra Trees Regressor
- Bagging Regressor
- Neural Net

# Linear Regression

In [8]:
# Train

linreg = LinearRegression().fit(X_train, y_train)

# Validate

y_hat_linreg = linreg.predict(X_val)
rmse_linreg = mean_squared_error(y_val, y_hat_linreg, squared=False)

print('Linear Regression Validation rmse: {:.3}'.format(rmse_linreg))

Linear Regression Validation rmse: 4.1e+06


# Random Forest

In [9]:
# Train

val_split_indices = [-1 if x in X_train.index else 0 for x in X_housing.index]
ps = PredefinedSplit(test_fold=val_split_indices)

estimator = RandomForestRegressor()
params = {'n_estimators': [25, 50, 100],
          'max_depth': [5, 10, 25, 50],
          'min_samples_split': [2],
          'min_samples_leaf': [1],
          'criterion': ["squared_error"],
          'max_features': [1],
          'bootstrap': [True, False],
          'random_state': [RANDOM_CONTROL]}
model_rf = GridSearchCV(estimator=estimator,
                     param_grid=params,
                     cv=ps)
model_rf.fit(X_housing, y_housing)

print('Random Forest Best Parameters: {}'.format(model_rf.best_params_))

# Validate

y_hat_rf = model_rf.predict(X_val)
rmse_rf = mean_squared_error(y_val, y_hat_rf, squared=False)

print('Random Forest Validation rmse: {:.3}'.format(rmse_rf))

Random Forest Best Parameters: {'bootstrap': True, 'criterion': 'squared_error', 'max_depth': 50, 'max_features': 1, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'random_state': 42}
Random Forest Validation rmse: 1.39e+06


# Gradient Boosting

In [10]:
# Train

estimator = GradientBoostingRegressor()
params = {'n_estimators': [25, 50, 100],
          'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
          'max_depth': [5, 10, 25, 50],
          'min_samples_split': [2],
          'min_samples_leaf': [1],
          'criterion': ["squared_error", "friedman_mse"],
          'max_features': [1],
          'random_state': [RANDOM_CONTROL]}
model_gb = GridSearchCV(estimator=estimator,
                     param_grid=params,
                     cv=ps)
model_gb.fit(X_housing, y_housing)

print('Random Forest Best Parameters: {}'.format(model_gb.best_params_))

# Validate

y_hat_gb = model_gb.predict(X_val)
rmse_gb = mean_squared_error(y_val, y_hat_gb, squared=False)

print('Gradient Boosting Validation rmse: {:.3}'.format(rmse_gb))

Random Forest Best Parameters: {'criterion': 'squared_error', 'learning_rate': 0.1, 'max_depth': 10, 'max_features': 1, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'random_state': 42}
Gradient Boosting Validation rmse: 1.17e+06


# AdaBoost

In [11]:
base_estimator = DecisionTreeRegressor()
estimator = AdaBoostRegressor(base_estimator=base_estimator)
params = {'base_estimator__max_depth': [5, 10, 25, 50],
          'base_estimator__splitter': ['best', 'random'],
          'n_estimators': [25, 50, 100],
          'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
          'random_state': [RANDOM_CONTROL]}
model_ab = GridSearchCV(estimator=estimator,
                     param_grid=params,
                     cv=ps)
model_ab.fit(X_housing, y_housing)

print('AdaBoost Best Parameters: {}'.format(model_ab.best_params_))

# Validate

y_hat_ab = model_ab.predict(X_val)
rmse_ab = mean_squared_error(y_val, y_hat_ab, squared=False)

print('AdaBoost Validation rmse: {:.3}'.format(rmse_ab))

AdaBoost Best Parameters: {'base_estimator__max_depth': 10, 'base_estimator__splitter': 'best', 'learning_rate': 0.5, 'n_estimators': 100, 'random_state': 42}
AdaBoost Validation rmse: 1.68e+06


# Extra Trees Regressor

In [13]:
estimator = ExtraTreesRegressor()
params = {'n_estimators': [25, 50, 100],
          'max_depth': [5, 10, 25, 50],
          'min_samples_split': [2],
          'min_samples_leaf': [1],
          'criterion': ["squared_error", "friedman_mse"],
          'max_features': [1],
          'random_state': [RANDOM_CONTROL]}
model_et = GridSearchCV(estimator=estimator,
                     param_grid=params,
                     cv=ps)
model_et.fit(X_housing, y_housing)

print('Extra Trees Best Parameters: {}'.format(model_et.best_params_))

# Validate

y_hat_et = model_et.predict(X_val)
rmse_et = mean_squared_error(y_val, y_hat_et, squared=False)

print('Extra Trees Validation rmse: {:.3}'.format(rmse_et))

Extra Trees Best Parameters: {'criterion': 'squared_error', 'max_depth': 10, 'max_features': 1, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 25, 'random_state': 42}
Extra Trees Validation rmse: 2.86e+06


# Bagging Regressor

In [14]:
base_estimator = DecisionTreeRegressor()
estimator = BaggingRegressor(base_estimator=base_estimator)
params = {'n_estimators': [25, 50, 100],
          'max_features': [1],
          'random_state': [RANDOM_CONTROL]}
model_br = GridSearchCV(estimator=estimator,
                     param_grid=params,
                     cv=ps)
model_br.fit(X_housing, y_housing)

print('Bagging Best Parameters: {}'.format(model_br.best_params_))

# Validate

y_hat_br = model_br.predict(X_val)
rmse_br = mean_squared_error(y_val, y_hat_br, squared=False)

print('Bagging Regressor Validation rmse: {:.3}'.format(rmse_br))

Bagging Best Parameters: {'max_features': 1, 'n_estimators': 100, 'random_state': 42}
Bagging Regressor Validation rmse: 3.05e+06


# Neural Net

In [ ]:
X_housing = X_housing.to_numpy()
y_housing = y_housing.to_numpy()
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()

#print(X_train.shape)
#print(y_train.shape)
train_dataloader = DataLoader([ [X_train[i], y_train[i]] for i in range(len(X_train)) ], batch_size=NN_BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
# Define MLP architecture
class Model(nn.Module):
    
    def __init__(self, device='cpu'):
        super(Model, self).__init__()
        self.device = device
        
        # Modify accordingly
        self.fc1 = nn.Linear(1, 4)
        self.fc2 = nn.Linear(4, 16)
        self.fc3 = nn.Linear(16, 64)
        self.fc4 = nn.Linear(64, 4)
        self.fc5 = nn.Linear(4, 1)
        self.relu = nn.ReLU()
        
        self.dense = nn.Sequential(self.fc1, self.relu, self.fc2, self.relu, self.fc3, self.relu, 
                                   self.fc4, self.relu, self.fc5)
        
    def forward(self, x):
        pred = self.dense(x)
        return pred
    
# Train
device = 'cpu'
model = Model()
optimizer = optim.Adam(model.parameters(), NN_LEARNING_RATE)
criterion = nn.MSELoss()
model.to(device)
for epoch in range(NN_NUM_EPOCHS):
    running_loss = 0
    for idx, (x_features, y_labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        x_features = x_features.to(device, dtype=torch.float)
        y_labels = y_labels.to(device, dtype=torch.float)
        prediction = model(x_features)
        loss = torch.sqrt(criterion(prediction, y_labels)) # Standardize RMSE loss
        loss.backward()
        optimizer.step()
        running_loss += loss
        if (idx+1) %100 == 0: 
            running_loss = format(running_loss/100, '.4f')
            print(f"Epoch [{epoch+1} Batches processed | {idx}] Loss: {running_loss}")
            running_loss = 0
print("Finished Training.")


In [ ]:
# Validate
X_features = torch.from_numpy(X_val)
y_labels = torch.from_numpy(y_val)
X_features = X_features.to(device, dtype=torch.float)
y_labels = y_labels.to(device, dtype=torch.float)
prediction = model(X_features)
rmse_nn = torch.sqrt(criterion(prediction, y_labels))

print('Neural Net Validation rmse: {:.3}'.format(rmse_nn))

**Wrap-up**

(Do this only before submitting to Kaggle; train over the entire set here after the hyperparameters are identified; then perform testing and submit results)

In [ ]:
df = pd.read_csv('data/test.csv') 
# Pre-process similar to above. Need to refactor into a function.

# Linear Regression
linreg = LinearRegression().fit(X_housing, y_housing)
y_hat_linreg = linreg.predict(X_val)

# Random Forest

# Gradient Boosting

# AdaBoost

# Neural Net

**Do not execute**

(Add misc. code here that was not utilized)